In [1]:
import pandas as pd
import numpy as np
import os
import requests
import env
import wrangle as w
import explore as exp

In [2]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

In [3]:
def extract_street_name(address):
    parts = address.split()
    
    for part in reversed(parts):
        if not part.isnumeric():
            return part
    
    return None

address = "2911 86 STREET"
street_name = extract_street_name(address)

print(street_name)


STREET


In [4]:
# Example usage:
app_token = env.app_token
year_to_retrieve = '2022'
max_req = 2000  # Specify the maximum number of observations to retrieve


In [5]:
df = w.wrangle_coll_stage1(year_to_retrieve, app_token)
df.head()

CSV file for 2022 already exists. Loading data from the CSV.


,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-01-01 04:30:00,2022-01-01,4:30,4491626,NaN,NaN,NaN,Southern parkway,NaN,NaN,0,0,0,0,Sedan,Pavement Slippery,Sedan,Unspecified
1,2022-01-01 05:17:00,2022-01-01,5:17,4491857,40.746930,-73.84866,"{'latitude': '40.74693', 'longitude': '-73.848...",GRAND CENTRAL PKWY,NaN,NaN,1,0,1,0,Sedan,Passing or Lane Usage Improper,Sedan,Unsafe Lane Changing
2,2022-01-01 01:30:00,2022-01-01,1:30,4491344,40.819157,-73.96038,"{'latitude': '40.819157', 'longitude': '-73.96...",HENRY HUDSON PARKWAY,NaN,NaN,0,0,0,0,Sedan,Unspecified,Station Wagon/Sport Utility Vehicle,NaN
3,2022-01-01 16:40:00,2022-01-01,16:40,4491478,40.806107,-73.91799,"{'latitude': '40.806107', 'longitude': '-73.91...",NaN,BRONX,10454.0,0,0,0,0,Sedan,Unspecified,Sedan,Unspecified
4,2022-01-01 10:00:00,2022-01-01,10:00,4491791,NaN,NaN,NaN,VANDAM STREET,NaN,NaN,1,0,1,0,Station Wagon/Sport Utility Vehicle,Driver Inattention/Distraction,Sedan,Unspecified


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54214 entries, 0 to 58536
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   crash_datetime                 54214 non-null  datetime64[ns]
 1   crash_date                     54214 non-null  datetime64[ns]
 2   crash_time                     54214 non-null  object        
 3   collision_id                   54214 non-null  int64         
 4   latitude                       48263 non-null  float64       
 5   longitude                      48263 non-null  float64       
 6   location                       48263 non-null  object        
 7   on_street_name                 41609 non-null  object        
 8   borough                        34461 non-null  object        
 9   zip_code                       34453 non-null  float64       
 10  number_of_persons_injured      54214 non-null  int64         
 11  number_of_persons_ki

In [7]:
# Example usage:
info_dataframe = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,45821,datetime64[ns]
1,crash_date,0,0.0,0,0,365,datetime64[ns]
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,54204,int64
4,latitude,5951,11.0,0,0,23107,float64
5,longitude,5951,11.0,0,0,19572,float64
6,location,5951,11.0,0,0,27538,object
7,on_street_name,12605,23.0,0,0,3509,object
8,borough,19753,36.0,0,0,5,object
9,zip_code,19761,36.0,0,0,200,float64


In [8]:
missing_df = df[(df['latitude'].notnull()) & (df[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]


In [10]:
len(missing_df)

27095

___
We have identified any row with valid lat and long and missing 'zip_code', 'borough', 'on_street_name'.
___

We can now run make api calls to google geocoding and get the missing information using the lat and long. 

In [32]:
import pandas as pd
import requests

def fill_missing_geocode_info(df, api_key, save_path, save_interval=1000):
    # Create a list to store geocode data
    geocode_data = []
    
    # Initialize a counter to keep track of API calls
    api_call_count = 0
    
    # Iterate through rows of the original DataFrame
    for index, row in df.iterrows():
        # Check if ZIP code, borough, and on-street name are missing
        if pd.isna(row['zip_code']) or pd.isna(row['borough']) or pd.isna(row['on_street_name']):
            lat = row['latitude']
            lng = row['longitude']
            
            # Make the API request
            url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{lng}&key={api_key}"
            response = requests.get(url)
            
            # Check if the request was successful
            if response.status_code == 200:
                data = response.json()
                
                # Initialize variables to store the address components
                zipcode = None
                borough = None
                street_address = None
                
                # Parse the response to extract address components
                for result in data.get('results', []):
                    for component in result.get('address_components', []):
                        if 'postal_code' in component['types']:
                            zipcode = component['short_name']
                        if 'sublocality_level_1' in component['types']:
                            borough = component['long_name']
                        if 'route' in component['types']:
                            street_address = component['long_name']
                
                # Fill in missing information in the DataFrame
                if pd.isna(row['zip_code']):
                    df.at[index, 'zip_code'] = zipcode
                if pd.isna(row['borough']):
                    df.at[index, 'borough'] = borough
                if pd.isna(row['on_street_name']):
                    df.at[index, 'on_street_name'] = street_address
                
                # Append geocode data to the list
                geocode_data.append({'latitude': lat, 'longitude': lng, 'geocode_json': data})
                
                # Increment the API call counter
                api_call_count += 1
                
                # Check if it's time to save progress
                if api_call_count % save_interval == 0:
                    # Save the DataFrame and geocode data to CSV files
                    df.to_csv(save_path, index=False)
                    geocode_df = pd.DataFrame(geocode_data)
                    geocode_df.to_csv(f'geocode_data_{api_call_count}.csv', index=False)
                    print(f"Progress saved after {api_call_count} API calls.")
            else:
                print(f"Error: {response.status_code} for row {index}")
    
    # Save the final DataFrame and geocode data to CSV files
    df.to_csv(save_path, index=False)
    geocode_df = pd.DataFrame(geocode_data)
    geocode_df.to_csv(f'geocode_data_{api_call_count}.csv', index=False)
    
    return df

Make the api calls and update the columns

In [33]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geo_df_1.csv'

In [ ]:
# Fill missing geocode information in the DataFrame
first_run = fill_missing_geocode_info(missing_df, api_key, save_path, save_interval=1000)


Optional: if the fill_missing_geocode_info function stalls or otherwise fails to load the dataframe to filled_df, then we can load the csv files created by fill_missing_geocode_info as a safety mechanism (save_interval). 

In [97]:
first_run = pd.read_csv('geo_df_1.csv')

In [91]:
len(first_run)

27095

Lets see the new info


In [92]:
# Example usage:
info_dataframe = exp.create_info_dataframe(first_run)  # Assuming data_df is your DataFrame
info_dataframe

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,1341,5.0,0,0,4312,object
8,borough,1783,7.0,0,0,10,object
9,zip_code,1783,7.0,0,0,205,float64


for some reason we have a difference in nulls on street name, borough and zip_code despite running the api calls.

we should have a maximum of 5951 missing street name, borough and zip_code. Yet we have 5972 missing zip codes.  

update: I found out that the fill_missing_geocode_info function had stalled out and did not get all of the missing geocode info. The discrepancy in null values form lat and long and the street name, borough and zip_code is caused by this. I will have to create a dataframe containing rows with lat and long that are missing any street name, borough and zip_code. Run that df through the fill_missing_geocode_info and merge the first run with the second run. 

the street name has 2081 which is lower than I expected as I seem to remember checking and finding out that all rows with missing lat and long also were missing street name. I will have to double check this later.

In [93]:
missing_first_run = first_run[(first_run['latitude'].notnull()) & (first_run[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]

In [94]:
len(missing_first_run)

3081

we ill have to run missing_first_run through fill_missing_geocode_info to get the missing geo location info. 

In [34]:
second_run = fill_missing_geocode_info(missing_first_run, api_key, save_path, save_interval=1000)

Progress saved after 1000 API calls.
Progress saved after 2000 API calls.
Progress saved after 3000 API calls.


In [96]:
second_run = pd.read_csv('geo_df_2.csv')

Lets confirm that we filled the missing info

In [98]:
second_run.head(1)

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,NaN,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


Lets now load the first run df with the second run df

In [99]:
len(first_run), len(second_run)

(27095, 3081)

lets print the null count of the first run so we can compare it later

now we must concat the 2 dataframes but first we need to remove any rows that exist in the filled_df (second api run) from the first_run dataframe. 

In [110]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = second_run['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
first_run = first_run[~first_run['collision_id'].isin(collision_ids_to_remove)]

# Concatenate the two dataframes
combined_one_tow = pd.concat([second_run, first_run], ignore_index=True)

In [111]:
len(first_run)

24014

In [113]:
len(combined_one_tow)

27095

In [114]:
# Example usage:
filled_df_info = exp.create_info_dataframe(combined_one_tow)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,0,0.0,0,0,4430,object
8,borough,1,0.0,0,0,10,object
9,zip_code,0,0.0,0,0,205,float64


In [115]:
missing_df_second = combined_one_tow[(combined_one_tow['latitude'].notnull()) & (combined_one_tow[['zip_code', 'borough', 'on_street_name']].isna().any(axis=1))]

In [125]:
len(missing_df_second)

1

In [126]:
missing_df_second

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,NaN,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [109]:
# Replace 'YOUR_API_KEY' with your actual Google Geocoding API key
api_key = env.google_api_key 

# Define the path to save the geocode data CSV file
save_path = 'geo_df_3.csv'

In [127]:
third_run = fill_missing_geocode_info(missing_df_second, api_key, save_path, save_interval=1000)

In [119]:
third_run = pd.read_csv('geo_df_3.csv')

In [135]:
third_run

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [129]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = third_run['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
combined_one_tow = combined_one_tow[~combined_one_tow['collision_id'].isin(collision_ids_to_remove)]

# Concatenate the two dataframes
combined_three = pd.concat([third_run, combined_one_tow], ignore_index=True)

In [151]:
len(combined_three)

27095

In [153]:
combined_three.to_csv('final_geo_filled.csv')

In [152]:
# Example usage:
filled_df_info = exp.create_info_dataframe(combined_three)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,24571,object
1,crash_date,0,0.0,0,0,365,object
2,crash_time,0,0.0,0,0,1432,object
3,collision_id,0,0.0,0,0,27091,int64
4,latitude,0,0.0,0,0,16518,float64
5,longitude,0,0.0,0,0,14373,float64
6,location,0,0.0,0,0,18602,object
7,on_street_name,0,0.0,0,0,4430,object
8,borough,1,0.0,0,0,10,object
9,zip_code,0,0.0,0,0,205,float64


In [ ]:
combined_three[combined_three['collision_id'] == 4562046]

,crash_datetime,crash_date,crash_time,collision_id,latitude,longitude,location,on_street_name,borough,zip_code,number_of_persons_injured,number_of_persons_killed,number_of_motorist_injured,number_of_motorist_killed,vehicle_type_code1,contributing_factor_vehicle_1,vehicle_type_code2,contributing_factor_vehicle_2
0,2022-08-31 20:00:00,2022-08-31,20:00,4562046,40.85169,-73.95238,"{'latitude': '40.85169', 'longitude': '-73.952...",CROSS BRONX EXPY,None,7024.0,0,0,0,0,Station Wagon/Sport Utility Vehicle,Following Too Closely,Station Wagon/Sport Utility Vehicle,Unspecified


In [142]:
len(df), len(combined_three)

(54214, 27095)

In [143]:
# List of unique collision_ids in filled_df
collision_ids_to_remove = combined_three['collision_id'].unique()

# Remove rows from first_run where collision_id matches any value in collision_ids_to_remove
df = df[~df['collision_id'].isin(collision_ids_to_remove)]


In [144]:
len(df)

27119

In [145]:
# Concatenate the two dataframes
df = pd.concat([combined_three, df], ignore_index=True)

In [146]:
len(df)

54214

In [150]:
df.to_csv('df_geo_cleaned.csv')

In [147]:
# Example usage:
filled_df_info = exp.create_info_dataframe(df)  # Assuming data_df is your DataFrame
filled_df_info

,Column,Null_Count,Null_Percentage,Zero_Count,Blank_Count,Unique_Values,Data_Type
0,crash_datetime,0,0.0,0,0,49399,object
1,crash_date,0,0.0,0,0,730,object
2,crash_time,0,0.0,0,0,1440,object
3,collision_id,0,0.0,0,0,54204,int64
4,latitude,5951,11.0,0,0,23107,float64
5,longitude,5951,11.0,0,0,19572,float64
6,location,5951,11.0,0,0,27538,object
7,on_street_name,740,1.0,0,0,6035,object
8,borough,4188,8.0,0,0,10,object
9,zip_code,4187,8.0,0,0,212,float64


filled_df contains only the filtered df rows with missing location columns. 
we have to find a way to merge the filled df with the original df while keeping the updated values from filled df.

